# WS 01

In [ ]:
from sklearn.datasets import load_breast_cancer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

cancer = load_breast_cancer()

In [ ]:
cancer.keys()

In [ ]:
cancer.target_names

In [ ]:
df = pd.DataFrame(cancer.data[:,0:2], columns=cancer.feature_names[0:2])

df['class'] = cancer.target

df.sample(5, random_state=1)
# df.head()

convert class to string

In [ ]:
# df['class'] = np.where(df['class']==0, 'yes', 'no')  
# df['class'] = np.where(df['class']==0, 'malignant', 'benign') 

target_map = {0:'malignant', 1:'benign'}  # same
df['class'] = df['class'].map(target_map)

# df.head()
df.sample(5, random_state=1)

In [ ]:
df.head()
# df.tail()

In [ ]:
df.isnull().sum()

In [ ]:
plt.figure(figsize=(5.2, 4))

sns.scatterplot(x='mean radius', y='mean texture', data=df, hue='class', style='class', 
                alpha=0.9, edgecolor='w', s=80)

plt.show()

In [ ]:
class_names, y = np.unique(df['class'], return_inverse=True)
class_names

In [ ]:
y[40:50]

In [ ]:
X = df.drop('class', axis=1)
X.head()

In [ ]:
# y = df['class']
np.unique(y)
y[:5]

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_sc = sc.fit_transform(X)
X_sc[:5]

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 3.5))

ax1.set_title('Before StandardScale (Raw data)')
sns.kdeplot(df['mean radius'], fill=True, ax=ax1)
sns.kdeplot(df['mean texture'], fill=True, ax=ax1)

ax2.set_title('After StandardScale')
sns.kdeplot(X_sc[:,0], fill=True, ax=ax2)
sns.kdeplot(X_sc[:,1], fill=True, ax=ax2)
sns.kdeplot(df['mean texture'], fill=True, ax=ax1)

plt.show()

In [ ]:
plt.scatter(X_sc[:,0][y==0], X_sc[:,1][y==0], marker='x')
plt.scatter(X_sc[:,0][y==1], X_sc[:,1][y==1], edgecolor='w', s=50)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=0.25, random_state=1) #

X_train.shape, X_test.shape

In [ ]:
y_train.shape, y_test.shape 

In [ ]:
X_train[:5]

In [ ]:
y_train[:5]

# ws 02

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense 

model = Sequential()

model.add(Dense(32, activation='relu', input_shape=(2,)))  

model.add(Dense(1, activation='sigmoid')) # 

In [ ]:
input_size = X_train.shape[-1]
input_size

In [ ]:
# input_size = X_train.shape[-1]
# model = Sequential([
#     Dense(32, activation='relu', input_shape=(input_size,)),
#     Dense(1, activation='sigmoid')
# ])

## Functional model

In [ ]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Model

inputs = Input(shape=(2,))

x = Dense(32, activation="relu")(inputs)

outputs = Dense(1, activation="relu")(x)

model = Model(inputs=inputs, outputs=outputs, name="my_model")
model.summary()

In [ ]:
model.summary()

In [ ]:
model.output_shape

In [ ]:
model.get_config()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Train

In [ ]:
import time

start = time.time()

history = model.fit(X_train, y_train, epochs=20, verbose=1, batch_size=32, validation_split=0.25)

end = time.time()

print(f"Runtime of the program is {end - start} secs")
print("Time Taken: {:.3f} secs".format(end - start))

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
y_pred_pr = model.predict(X_test)
y_pred_pr[:4]

In [ ]:
y_pred = (y_pred_pr > 0.5).astype(int)  
# y_pred = np.where(y_pred_pr > .5, 1,0)

y_pred[:4]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print('Score->',score[1])

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
df_hist = pd.DataFrame.from_dict(history.history)
history.params
history.history.keys()

In [ ]:
plt.figure(figsize=(4.7, 3.5))

df_hist['loss'].plot(style='b--', label='Trainning')
df_hist['val_loss'].plot(style='r-', label='Test')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.legend()
plt.title('Loss')
plt.show()

In [ ]:
plt.figure(figsize=(4.7, 3.5))

df_hist['accuracy'].plot(style='b--', label='Trainning')
df_hist['val_accuracy'].plot(label='Test')

plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend()
plt.tight_layout() 

plt.show()

## Tensor board

In [ ]:
# from keras.callbacks import TensorBoard

# history = model.fit(X_train, y_train, epochs=30, verbose=0, 
#                     batch_size=32, validation_split=0.25,
#                    callbacks=[TensorBoard(log_dir='./log')])  


In [ ]:
# %load_ext tensorboard
# # %reload_ext tensorboard
# %tensorboard --logdir ./log

In [ ]:

from mlxtend.plotting import plot_decision_regions

ax = plot_decision_regions(X_test, y_test, clf=model)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, class_names, framealpha=0.5)

plt.title('Breast Cancer:') 
plt.xticks([])
plt.yticks([])
plt.xlabel('mean radius')
plt.ylabel('mean texture')

plt.show()

## Predict

In [ ]:
# X_new = [[16.4, 14.1]] #
X_new = [[16.4, 14.1],  # 
         [18.5, 15.1]] #

# X_new = X[104:110]

X_new_sc = sc.transform(X_new)
X_new_sc

In [ ]:
y_pred_pr = model.predict(X_new_sc)
y_pred = (y_pred_pr > 0.5).astype(int) 

print(class_names[y_pred])

In [ ]:
y_pred = y_pred.reshape(1,-1)[0] 

from mlxtend.plotting import plot_decision_regions

ax = plot_decision_regions(X_test, y_test, clf=model)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, class_names, framealpha=0.5)

plt.title('Breast Cancer: New data points')
plt.xticks([])
plt.yticks([])
plt.xlabel('mean radius')
plt.ylabel('mean texture')

plt.scatter(X_new_sc[:,0][y_pred==0], X_new_sc[:,1][y_pred==0], marker='s', s=180)
plt.scatter(X_new_sc[:,0][y_pred==1], X_new_sc[:,1][y_pred==1], marker='^', s=350) # 

plt.show()

# ws 03

In [ ]:
model.save('./mymodel.h5')

In [ ]:
model.save('./mymodel1')

# ws 04 

In [ ]:
import pickle
pickle.dump(sc, open('./scaler.pkl','wb'))

In [ ]:
file_name = "classname.pkl"

open_file = open(file_name, "wb")
pickle.dump(class_names, open_file)
open_file.close()

# ws 05

In [ ]:
from tensorflow.keras.models import load_model

my_model = load_model('./mymodel.h5') 

In [ ]:
my_model.summary()

In [ ]:
from tensorflow.keras.models import load_model

new_model = load_model('./mymodel1')

In [ ]:
model = new_model

In [ ]:
# List all weight tensors 
model.get_weights()

In [ ]:
import pickle
sc2 = pickle.load(open('./scaler.pkl','rb'))

In [ ]:
file_name = "classname.pkl"
open_file = open(file_name, "rb")
class_names = pickle.load(open_file)
open_file.close()

class_names

In [ ]:
X_new = [[16.4, 14.1]] # 
X_new = [[16.4, 14.1],  # 
         [18.5, 15.1],
         [17.7, 18.7]] #

X_new_sc = sc2.transform(X_new)

In [ ]:
y_pred_pr = my_model.predict(X_new_sc)
y_pred = (y_pred_pr > 0.5).astype(int) 
y_pred

In [ ]:
print(class_names[y_pred])

In [ ]:
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions

y = y_pred.reshape(1, -1)[0]

ax = plot_decision_regions(X_new_sc, y, clf=my_model)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, class_names, framealpha=0.5)

plt.title('Breast Cancer (from saved model)') 
# plt.xticks([])
plt.yticks([])
plt.xlabel('mean radius')
plt.ylabel('mean texture')

plt.scatter(X_new_sc[:,0][y==0], X_new_sc[:,1][y==0], marker='s', s=150, c='b')
plt.scatter(X_new_sc[:,0][y==1], X_new_sc[:,1][y==1], marker='^', s=150, c='r')
plt.show()